In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from loaders import get_datetime_cols
import re

In [2]:
sheet_url = st.secrets["private_gsheets_url"]

In [3]:
from connections import get_google_conn
import streamlit as st
import pandas as pd

gc = get_google_conn()

# Uses st.cache to only rerun when the query changes or after 10 min.
def get_worksheet(url, sheet_num=0):
    worksheet = gc.open_by_url(url).get_worksheet(sheet_num)
    df = pd.DataFrame(worksheet.get_all_values())
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.columns = [col.lower() for col in df.columns]
    return df

In [4]:
cheki_df = get_datetime_cols(get_worksheet(sheet_url, 0))
person_df = get_worksheet(sheet_url, 1)
#venue_df = get_worksheet(sheet_url, 2)

In [5]:
person_df

,entity,name1,group1,name2,group2,name3,group3,name4,group4,sns
1,勝月蘭丸@1682,勝月蘭丸,1682,,,,,,,
2,承和 薫@1682,承和 薫,1682,,,,,,,
3,月見夜零@1682,月見夜零,1682,,,,,,,
4,神山おだし@1682,神山おだし,1682,,,,,,,
5,天野ひろこ@.BPM,天野ひろこ,.BPM,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
580,,,,,,,,,,
581,,,,,,,,,,
582,,,,,,,,,,
583,,,,,,,,,,https://twitter.com/Polalight_hina


In [6]:
names_dict = {}
name_cols = [col for col in person_df.columns if "name" in col]
print(name_cols)
for idx, row in person_df.iterrows():
    name_group_list = []
    for col in name_cols:
        if row[col] != "":
            name_group_list.append({"name": row[col], "group": row[f"group{col[-1]}"]})
    names_dict[row["entity"]] = name_group_list
        

['name1', 'name2', 'name3', 'name4']


In [7]:
names_dict["Suzy"]

[{'name': 'Suzy', 'group': ''},
 {'name': 'Suzy', 'group': '夢☆ペディア'},
 {'name': 'Suzy Lee', 'group': '花ノ色リミデット'}]

In [8]:
groups = []
for value in names_dict.values():
    for item in value:
        if item['group'] not in groups:
            groups.append(item['group'])

In [9]:
for i in groups:
    print(i)

1682
.BPM
【eN】
Chou2Precede
#あんのわーるど
蜜兎
℃-ute
ぷらしゅが
＋Sugar.
花ノ色リミデット
37.2℃-ナナドニブ-
81moment
ACR*0
ACT102
AdFicTioN
AiDOLOXXXY
Aither
Ange☆Reve
ASH
Asterisk*zero
baby bear Party
BooN! BooN! BooooN
Brio Gaja
BSK
BUPPPLE
Chase×Chase
CHEQUEMATE
CHERRY GIRLS PROJECT
CODE:0000
Cosmic
CYCLONISTA
DESURABBITS
DopeDrop
DOUBLE
elsy
EVE研修生
excess
Finally
143∽
FinalSelection
FIRST☆STAR
G'ASH
GANGDEMIC
雷電
天風-AMAKAZE-
Honey Bae Bee
Ice Ink
iLiFE!
JAPANARIZM
La❀花ノたみ
Leaders
たまご姫【たまプリ】
Li-V-RAVE
白色のマフィア
Lil na Valley
LOVE IZ DOLL
Love Jammin
Lyrical school
Magical Magic
MyDearDarlin'
NINJA紅
OTHE11O
P-Loco
Parasite.Kiss
Phantomy
酔生夢死
PUNKY RAD PINK
惑星竜宮
ReverseTokyo
Revival:I
Riw
Rizbeat
Seishun Youth Academy
Seloi
SHOW!ME!機嫌
SPADES
Stella☆Blue
SwayEmotionsSlightly
ユレルランドスケープ
Teamくれれっ娘!
THAT's so
unichan
VARY GIRLS PROJECT
yokubary
Youぱ〜む
YUENI
Zero Project
アイオケ
Ry's
あっしゅない！
あにま〜る
あむちゃんず
アンシュア
うたた寝シエスタ
ウレテン
おはよう！もきゅれーしょん
ガラクタ！ぽんこつ？
きいろちゃんず
いちぜん!
きみのロラゼパム
キューっと！センチめんたる
くるーず
クレアシオン
こっぺぱんのみみ
さとりモンスター
サバトライム

In [33]:
from collections import defaultdict
counts = defaultdict(list)
for row in cheki_array:
    updated_row = list(filter(None, set(row)))
    row_size = len(updated_row)
    for name in updated_row:
        counts[name].append(row_size)

In [35]:
from collections import Counter

Counter(counts["Suzy"])

Counter({1: 942, 5: 5, 4: 5, 3: 20, 2: 21})

In [36]:
grouped_counts = defaultdict()
for key, value in counts.items():
    grouped_counts[key] = Counter(value)

In [41]:
names_df = pd.DataFrame(grouped_counts).T.fillna(0)

In [43]:
names_df['total'] = names_df.sum(axis=1)

In [57]:
names_df.sort_values(by="total", ascending=False)[:20][['total', 1, 2, 3, 4, 5, 6, 7, 8]].astype(int)

,total,1,2,3,4,5,6,7,8
Suzy,993,942,21,20,5,5,0,0,0
恵深あむ@あむみほ,754,717,26,2,2,5,2,0,0
七瀬千夏@たまご姫【たまプリ】,310,272,7,19,1,0,3,8,0
瀬乃悠月@恒星宇宙,273,244,23,3,0,2,1,0,0
天音ゆめ@たまご姫【たまプリ】,232,205,5,1,9,1,3,8,0
楠木りほ@天使突抜ニ読ミ,217,194,4,19,0,0,0,0,0
火野快飛@恒星宇宙,133,107,20,3,0,2,1,0,0
中谷亜優@たまご姫【たまプリ】,124,102,1,1,8,1,3,8,0
雅春奈@ユレルランドスケープ,123,106,8,6,3,0,0,0,0
椎名まどか@たまご姫【たまプリ】,123,99,2,1,9,1,3,8,0


In [95]:
def group_cheki_by_name(df):
    name_cols = [col for col in df.columns if "name" in col]
    chekis = []
    for idx, row in df.iterrows():
        name_count = len(list(filter(None, row[name_cols].values)))
        for col in df.columns:
            if (row[col] is not np.nan) and ("name" in col) and (row[col]):
                chekis.append(
                    {
                        "cheki_id": idx,
                        "date": row["date"].date(),
                        "person": row[col],
                        "location": row["location"],
                        "year": row["date"].year,
                        "month": row["date"].month,
                        "name": split_name_group(row[col])[0],
                        "group": split_name_group(row[col])[1],
                        "n_shown": name_count, 
                    }
                )
    return pd.DataFrame(chekis)

In [96]:
export_df = group_cheki_by_name(cheki_df)

In [97]:
export_df[export_df["name"]=="Suzy"].groupby(["name", "n_shown"])['person'].count()

name  n_shown
Suzy  1          942
      2           21
      3           20
      4            5
      5            5
Name: person, dtype: int64

In [115]:
topdown = export_df.groupby(["name", "n_shown"])['person'].count().sort_index(level=0).reset_index()

In [116]:
topdown

,name,n_shown,person
0,AIRI,1,1
1,Aarie,1,1
2,Ale 愛礼,1,4
3,Ally,1,7
4,Ally,2,2
...,...,...,...
533,鳴上綺羅,6,1
534,黒ゑリズ,1,1
535,黒木結愛,1,2
536,黒瀬瑠衣,1,25


In [125]:
wide_df = topdown.pivot(index="name", columns="n_shown", values="person").fillna(0).astype(int)

In [126]:
wide_df['total'] = wide_df.sum(axis=1)

In [129]:
wide_df = wide_df[['total'] + [i for i in range(1,9)]]

In [148]:
totals = wide_df.sort_values(by=["total"], ascending=False).reset_index()

In [155]:
totals.index.name = "yay"

In [159]:
totals

n_shown,name,total,1,2,3,4,5,6,7,8
yay,,,,,,,,,,
0,Suzy,993,942,21,20,5,5,0,0,0
1,恵深あむ,754,717,26,2,2,5,2,0,0
2,七瀬千夏,310,272,7,19,1,0,3,8,0
3,瀬乃悠月,273,244,23,3,0,2,1,0,0
4,天音ゆめ,232,205,5,1,9,1,3,8,0
...,...,...,...,...,...,...,...,...,...,...
383,柊・D・ナナ,1,1,0,0,0,0,0,0,0
384,やぎぬま,1,1,0,0,0,0,0,0,0
385,みゆりん,1,1,0,0,0,0,0,0,0


In [1]:
person_df

NameError: name 'person_df' is not defined